In [157]:
pip install langchain_chroma

In [158]:
!pip install pydantic

In [159]:
!pip install langchain-cohere

In [160]:
from openai import OpenAI
from getpass import getpass
import re
import json
import numpy as np
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain_cohere import CohereRerank
from langchain.retrievers import ContextualCompressionRetriever

In [161]:
MY_API_KEY = getpass("OpenAI API Key :")

In [ ]:
# OpenAI API 클라이언트 객체 생성
client = OpenAI(api_key=MY_API_KEY)


In [163]:
with open('data/CHK_cards_data.json', 'r', encoding='utf-8') as f:
    # json.load : JSON 데이터 불러오기
    card_list = json.load(f)

print(len(card_list))
card_list[:10]

352


[{'name': 'ONE 체크카드',
  'cate_txt': '체크',
  'corp_txt': '케이뱅크',
  'benefits': ['[K-패스] 매일 대중교통 타고 교통 요금의 최대 53%까지 돌려받아요',
   '대중교통 혜택 추가 3천원 캐시백',
   '[모두 다 캐시백] 오프라인 0.7%, 온라인 1.2% 캐시백',
   '[여기서 더 캐시백] 자주 쓰는 브랜드에서 최대 7% 캐시백',
   '[369 캐시백] 3번 결제할 때마다 1,000원 캐시백',
   '기본 혜택'],
  'page_url': 'https://www.card-gorilla.com/card/detail/2749'},
 {'name': '노리2 체크카드(KB Pay)',
  'cate_txt': '체크',
  'corp_txt': 'KB국민카드',
  'benefits': ['[일상] 스타벅스, 커피빈 10% 할인',
   '[일상] 구글플레이스토어, 앱스토어 10% 할인',
   '[일상] 인터파크 티켓 10% 할인',
   '[일상] 올리브영, 미용실 업종 5% 할인',
   '[일상] GS25, CU 5% 할인',
   '[일상]넷플릭스, 유튜브프리미엄 1,000원 할인',
   '[일상] 배달의 민족, 요기요 1,000원 할인',
   '[일상] SKT, KT, LG U+, Liiv M 2,500원 할인',
   '[일상] CGV 4,000원 할인',
   '[일상] 에버랜드, 롯데월드 15,000원 할인',
   '[KB Pay] 오프라인 가맹점 2% 추가 할인',
   '[KB Pay] 온라인 가맹점 2% 추가 할인'],
  'page_url': 'https://www.card-gorilla.com/card/detail/2422'},
 {'name': '신한카드 Deep Dream 체크(미니언즈)',
  'cate_txt': '체크',
  'corp_txt': '신한카드',
  'benefits': ['[모두드림] 국내/외 일시불+할부 이용 가맹점 0.2% 기본

In [164]:
with open('data/CRD_cards_data.json', 'r', encoding='utf-8') as f:
    # json.load : JSON 데이터 불러오기
    card_list = json.load(f)

print(len(card_list))
card_list[:10]

769


[{'name': '신한카드 Mr.Life',
  'cate_txt': '신용',
  'corp_txt': '신한카드',
  'benefits': ['월납요금(공과금) 10% 할인서비스',
   '편의점 10% 할인',
   '병원/약국업종 10% 할인',
   '세탁소 업종 10% 할인',
   '온라인 쇼핑 10% 할인 ',
   '택시 10% 할인 ',
   '식음료 10% 할인',
   '3대 마트 10% 할인',
   '4대 정유사 리터당 60원 할인',
   ' 인테이크몰 20% 할인'],
  'page_url': 'https://www.card-gorilla.com/card/detail/13'},
 {'name': '삼성카드 taptap O',
  'cate_txt': '신용',
  'corp_txt': '삼성카드',
  'benefits': ['라이프스타일 패키지 선택',
   '대중교통·택시 10% 결제일할인',
   '이동통신요금 10% 결제일할인',
   'CGV 및 롯데시네마 5,000원 결제일할인',
   '해외 1.3% 적립'],
  'page_url': 'https://www.card-gorilla.com/card/detail/51'},
 {'name': '삼성카드 & MILEAGE PLATINUM (스카이패스)',
  'cate_txt': '신용',
  'corp_txt': '삼성카드',
  'benefits': ['모든 가맹점 이용금액 1,000원당 1마일리지 기본적립',
   '(특별적립) 국내형: 백화점 이용금액 1,000원당 (스카이패스) 1 마일리지 추가 적립',
   '(특별적립) 국내형: 주유 이용금액 1,000원당 (스카이패스) 1 마일리지 추가 적립',
   '(특별적립) 국내형: 커피 이용금액 1,000원당 (스카이패스) 1 마일리지 추가 적립',
   '(특별적립) 국내형: 편의점 이용금액 1,000원당 (스카이패스) 1 마일리지 추가 적립',
   '(특별적립) 국내형: 택시 이용금액 1,000원당 (스카이패스

In [ ]:
# JQ 필터로 문자열을 설정하여 JSON 데이터를 변환
jq_schema = (
    ".[] | {"
    "name: .name, "  # 카드 이름 필드 추출
    "category: .cate_txt, "  # 카드 카테고리 필드 추출
    "corporation: .corp_txt, "  # 카드 발급사 필드 추출
    "benefits: (.benefits | map(tostring) | join(\", \")), "  # 여러 혜택 목록을 join(", ")을 사용해 하나의 문자열로 변환하여 저장
    "page_url: .page_url"  # 카드 상세 페이지 URL 필드 추출
    "} | tostring"  # 최종적으로 JSON 객체를 문자열로 변환
)

# 여러 JSON 파일을 로드하여 데이터 처리
loaders = [
    JSONLoader("data/CHK_cards_data.json", text_content=True, jq_schema=jq_schema),  # 체크카드 데이터 로드
    JSONLoader("data/CRD_cards_data.json", text_content=True, jq_schema=jq_schema)  # 신용카드 데이터 로드
]




In [166]:
# Recursive splitter 객체 생성
json_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,
                                              chunk_overlap = 50
                                              )

In [167]:
my_chunks = []  # 청크를 보관할 리스트

for loader in loaders:
    # 데이터 로드 및 분할
    my_chunks.extend(loader.load_and_split(json_splitter))  # json_splitter를 사용하여 텍스트 분할

print("청크 개수 :", len(my_chunks))
my_chunks


청크 개수 : 1121


[Document(metadata={'source': 'C:\\Users\\host\\Desktop\\PROJECT_SeSAC_03\\data\\CHK_cards_data.json', 'seq_num': 1}, page_content='{"name":"ONE 체크카드","category":"체크","corporation":"케이뱅크","benefits":"[K-패스] 매일 대중교통 타고 교통 요금의 최대 53%까지 돌려받아요, 대중교통 혜택 추가 3천원 캐시백, [모두 다 캐시백] 오프라인 0.7%, 온라인 1.2% 캐시백, [여기서 더 캐시백] 자주 쓰는 브랜드에서 최대 7% 캐시백, [369 캐시백] 3번 결제할 때마다 1,000원 캐시백, 기본 혜택","page_url":"https://www.card-gorilla.com/card/detail/2749"}'),
 Document(metadata={'source': 'C:\\Users\\host\\Desktop\\PROJECT_SeSAC_03\\data\\CHK_cards_data.json', 'seq_num': 2}, page_content='{"name":"노리2 체크카드(KB Pay)","category":"체크","corporation":"KB국민카드","benefits":"[일상] 스타벅스, 커피빈 10% 할인, [일상] 구글플레이스토어, 앱스토어 10% 할인, [일상] 인터파크 티켓 10% 할인, [일상] 올리브영, 미용실 업종 5% 할인, [일상] GS25, CU 5% 할인, [일상]넷플릭스, 유튜브프리미엄 1,000원 할인, [일상] 배달의 민족, 요기요 1,000원 할인, [일상] SKT, KT, LG U+, Liiv M 2,500원 할인, [일상] CGV 4,000원 할인, [일상] 에버랜드, 롯데월드 15,000원 할인, [KB Pay] 오프라인 가맹점 2% 추가 할인, [KB Pay] 온라인 가맹점 2% 추가 할인","page_url":"https://www.card-gorilla.c

In [ ]:
# my_embedding = OpenAIEmbeddings(model="text-embedding-3-small",
#                                api_key=MY_API_KEY
#                                )

In [169]:
# # DB 생성 경로 설정
# my_directory = "VectorStores/"

# # from_documents : 청킹된 문서(document객체)의 정보 및 임베딩, 경로정보로 Chroma DB 생성
# vectordb = Chroma.from_documents(documents = my_chunks,
#                                 embedding = my_embedding,
#                                 persist_directory = my_directory
#                                 )

### 생성된 DB 불러와서 사용하기

In [170]:
my_embedding = OpenAIEmbeddings(model='text-embedding-3-small',
                                openai_api_key=MY_API_KEY)
my_directory = "VectorStores/"

# 기존 생성된 db를 불러올 때는 embedding_function에 임베딩 객체 넣기
vectordb = Chroma(persist_directory = my_directory,
                                embedding_function=my_embedding
                                )

In [171]:
vectordb._collection.count()

1121

In [172]:
vectordb._collection.get()

{'ids': ['4f8d75dd-5057-4dad-aa46-993575fb0354',
  '026ebca1-b7f3-4cdd-a387-c7f625db642e',
  '9213b69b-bee4-415d-8ec9-3d5750c9ca28',
  '671c0426-d4e7-4e8c-b883-0b86ebcdfff2',
  '39834c5e-cdf1-44a0-aed2-a73ee1fcfac2',
  '794618ef-7e3b-4430-b428-a4356fcc2f03',
  'b4c3e10f-dc67-42f5-b945-b1fb8b0d52df',
  '151171d3-51e2-4bb0-a2b9-455e12f6cf90',
  '4e534e90-71f4-4db0-aa67-31aa3b7b8445',
  'db508d0e-2352-4d18-a9b6-0e63e073a804',
  '8b33ba8b-0b29-4b52-9fee-fcae83f693ce',
  'e41c3ed2-2acf-4555-ba87-6d990f864742',
  '67edfdb9-4e90-4da3-ab2c-3598a814540f',
  '707acb91-d689-4396-b1bb-a30a0eaa1cd2',
  'e9080f34-7242-4e99-970c-763e2fa03163',
  'd7a59109-7f67-4228-a95b-c282a5a90852',
  '4877f7a6-3293-4ca8-98de-316de8f3c8a9',
  'f22717e6-37b9-443a-96b8-9632af7520da',
  'fe5f142d-d376-4ffb-9c4d-d016343ea9d0',
  '44446d23-cad2-4c2c-bfd5-da84820b73b8',
  'bfcbedbf-7024-4bca-bca2-4499de21d9b2',
  'e72c6216-59b9-4f01-8b8b-4645698bb25b',
  'c548326f-77e1-4c9d-95dd-0d169db88938',
  '6f8d1e87-80a8-43cb-b352-

In [ ]:
# vectordb를 검색기(retriever)로 변환하여 사용할 수 있도록 as_retriever를 사용
my_retriever = vectordb.as_retriever(search_kwargs={"k":3}) #  search_kwargs : 검색할 유사도가 높은 청크와 최대 개수 지정(k값 디폴트는 4개)


In [ ]:
# 검색기에 MMR검색 타입으로 지정
# 유사도 높은 문서를 먼저 선택하여 가져와서 중복성을 줄이고 그 중에서 다양한 정보를 제공
my_retriever_mmr = vectordb.as_retriever(search_type="mmr",
                                         search_kwargs={"fetch_k":20, "k":10, "lambda_mult":0.5}
                                         # fetch_k : 벡터 DB에서 가져올 유사도가 높은 문서 수
                                         # k : 응답의 다양성을 고려하여 fetch_k 중에서 변환될 문서(fetch_k개 중에서 중복을 줄여줌)
                                         # lambda_mult : 0~1사이 실수 값으로 관련성과 다양성을 제어하는 파라미터
                                         # (0에 가까우면 다양성 우선, 1에 가까울수록 관련성 우선)
                                         )

In [249]:
chat_model = ChatOpenAI(model = "gpt-3.5-turbo",
                        api_key=MY_API_KEY
                        )

In [ ]:
# RetrievalQA 객체 생성 (문서 검색 기반 질문 응답 시스템)
# 검색된 문서를 바탕으로 질문에 대한 답변을 생성하는 문서 기반 QA 시스템
R_QA1 =RetrievalQA.from_chain_type(llm = chat_model,               # LLM모델 지정
                                   chain_type = "refine",          # 답변을 위해 청크들이 결합되는 타입 지정
                                   # stuff : 검색된 청크들의 내용을 간단히 결합하여 답변 생성(가장 기본)
                                   # map_reduce : 청크를 각각 처리 후, 개별 결과를 결합하여 답변 생성(문서의 규모가 클때 주로 사용)
                                   # refine : 이전 청크의 처리 결과를 다음 청크에 반영하여 지속적으로 응답을 개선한 후 답변 생성
                                   # map_rerank : 각 문서를 개별 평가하여 관련성을 점수화한 후, 가장 높은 문서를 선택하여 답변 생성
                                   retriever = my_retriever_mmr,       # MMR검색 타입으로 검색기 지정(우리는 DB까지 포함함)
                                   return_source_documents = True  # 어떤 문서를 참조했는지 출력
                                   )

#### ContextualCompressionRetriever

In [219]:
COHERE_API_KEY = getpass("Cohere API Key: ")

In [ ]:
# Cohere의 Re-ranking 모델을 활용하여 검색된 문서나 데이터를 가장 관련성이 높은 순으로 정렬하는 기능을 수행
my_rerank = CohereRerank(cohere_api_key = COHERE_API_KEY,
                        model = "rerank-v3.5", # cohere지원 reranking모델
                        top_n=10 # 검색된 문서 중 상위 10개 반환하도록 설정, 디폴트는 3개
                        ) 

In [ ]:
# ContextualCompressionRetriever 객체 설정
# 관련성이 높은 핵심 내용을 압축하여 간결하게 응답하기 위해 사용
compression_retriever = ContextualCompressionRetriever(base_compressor=my_rerank, # Cohere Re-ranking 모델을 활용한 문서 압축
                                                       base_retriever=my_retriever_mmr,  # mmr검색기를 기반으로 데이터 검색
                                                       )

In [253]:
R_QA2 = RetrievalQA.from_chain_type(llm=chat_model,
                                    chain_type = "refine",
                                    # 문맥압축검색기로 변화하여 설정
                                    retriever = compression_retriever,
                                    return_source_documents = True
                                    )

In [ ]:
persona1 = """
김형석 28세 남성
데이터 분석가를 꿈꾸는 취업준비생이다.
금융 도메인에 관심이 많다.
서울 거주중 부모님과 함께 산다.
수입은 파트 타임을 뛰면서 한달에 약 30~50만원에 정도 번다.
식사는 주로 집에서 해결한다.
차는 없고 대중교통을 주로 이용한다.
평소 카페에서 자격증 공부와 취업준비를 하고 있다.
취미는 클래식 음악 감상이다.
현재 불확실한 미래에 대해 고민이 많다.
"""

persona2 = """
이혜성 30세 여성
중견기업 입사 3년 차, 마케팅 부서 근무중이다.
서울 거주중이고 보증금 500만원 월세 50 만원의 방에서 자취한다. 
소득은 약 월 250~300만원 이다.
자차를 보유중이다.
취미로는 필라테스와 영화감상이 있다.
최근 주식 투자에 관심이 많다.
개인 일상 공유 블로그와 인스타그램을 운영중이다.
"""

persona3 = """
양호성 50세 남성
국내 1위 자동차기업에 다니며 직급은 부장이다.
지방광역시에 거주하며 40평 아파트와 고급 외제차 1대와 국산 중형 승용차 1대를 보유중이다.
수입은 약 월 900~1000만원 정도이다.
가부장적인 가장으로서 아내와 슬하에 딸 2명이 있다.
고민 사항으로 다가오는 정년퇴직과 점점 멀어지고 있는 자녀들과의 심리적 거리가 있다.
취미는 등산과 골프다.
비밀리에 맛집 공유 인스타그램을 운영중이며 20만 팔로워를 보유하고 있다.
"""

In [255]:
exemple1 = "김지훈 씨의 소비 패턴과 필요 혜택을 고려할 때, 다음과 같은 신용 및 체크카드를 추천드립니다.\n\n1. **신한카드 YOLO Tasty** (신용카드)\n   - **이유**: 이 카드는 대중교통 할인, 편의점 및 커피 전문점 할인, 그리고 배달앱 할인 혜택을 제공합니다. 또한, 연회비가 낮아 학생에게 부담이 적습니다. 다양한 생활 밀착형 혜택이 있어 김지훈 씨의 소비 패턴에 잘 맞습니다.\n\n2. **KB국민 청춘대로 톡톡 체크카드**\n   - **이유**: 이 체크카드는 대중교통 할인, 편의점 및 커피 전문점 할인, 그리고 통신비 자동이체 시 할인 혜택을 제공합니다. 연회비가 없고, 체크카드이기 때문에 신용카드 사용이 부담스러운 경우 적합합니다.\n\n3. **우리카드 D2 체크카드**\n   - **이유**: 이 카드는 대중교통 할인, 편의점 할인, 그리고 온라인 쇼핑몰 할인 혜택을 제공합니다. 특히, 배달앱 사용 시에도 할인이 적용되어 김지훈 씨의 소비 패턴에 잘 맞습니다.\n\n이 카드들은 김지훈 씨의 주요 소비 영역에서 혜택을 제공하며, 연회비가 없거나 낮아 학생에게 적합합니다. 각 카드의 세부 혜택과 조건을 비교하여 본인에게 가장 적합한 카드를 선택하는 것이 좋습니다. 또한, 카드 사용 시에는 항상 소비를 계획적으로 하여 신용 관리를 철저히 하는 것이 중요합니다."
exemple2 = "이수민 씨의 소비 패턴과 필요 혜택을 고려하여 몇 가지 신용 및 체크카드를 추천드리겠습니다.\n\n1. **신한카드 Deep Dream**:\n   - **이유**: 이 카드는 다양한 생활 영역에서 포인트 적립이 가능하며, 특히 커피 전문점, 대형마트, 온라인 쇼핑몰에서 높은 적립률을 제공합니다. 또한, 교통비 할인 혜택도 포함되어 있어 출퇴근 시 유용합니다.\n\n2. **삼성카드 taptap O**:\n   - **이유**: 커피 전문점과 레스토랑에서 할인 혜택이 있으며, 온라인 쇼핑몰에서도 할인 혜택을 제공합니다. 또한, 해외 결제 시 수수료 면제 혜택이 있어 해외 사용 시 유리합니다.\n\n3. **현대카드 ZERO**:\n   - **이유**: 무이자 할부 서비스가 제공되며, 모든 가맹점에서 기본 할인 혜택이 있습니다. 또한, 주유소 할인 혜택도 포함되어 있어 자동차 관련 지출에 유리합니다.\n\n4. **KB국민카드 탄탄대로 Miz&Mr 티타늄**:\n   - **이유**: 항공 마일리지 적립이 가능하며, 커피 전문점, 쇼핑몰, 주유소 등에서 할인 혜택을 제공합니다. 해외 결제 시에도 혜택이 있어 여행 시 유용합니다.\n\n5. **우리카드 DA@카드의정석**:\n   - **이유**: 온라인 쇼핑몰 할인과 커피 전문점 할인 혜택이 있으며, 교통비 할인도 가능합니다. 또한, 해외 결제 시 수수료 면제 혜택이 있어 해외 여행 시 유리합니다.\n\n이 카드들은 이수민 씨의 소비 패턴과 필요 혜택에 맞춰 다양한 할인과 적립 혜택을 제공하므로, 본인의 소비 습관에 가장 잘 맞는 카드를 선택하시면 좋겠습니다. 각 카드의 세부 혜택과 연회비를 비교하여 최적의 선택을 하시기 바랍니다."
exemple3 = "박준호 씨의 소비 패턴과 필요 혜택을 고려하여 몇 가지 신용카드와 체크카드를 추천드리겠습니다. 각 카드의 주요 혜택과 이유를 함께 설명드리겠습니다.\n\n1. **신한 The CLASSIC+ 카드**\n   - **공항 라운지 이용 및 여행자 보험**: 국내외 공항 라운지 무료 이용과 여행자 보험 혜택이 제공됩니다.\n   - **주유소 할인**: 주요 주유소에서 리터당 할인 혜택이 있습니다.\n   - **프리미엄 쇼핑 혜택**: 백화점 및 면세점에서 할인 혜택을 받을 수 있습니다.\n   - **마일리지 적립**: 항공사 마일리지 적립이 가능하여 해외 출장 시 유용합니다.\n   - **추천 이유**: 다양한 프리미엄 혜택과 마일리지 적립이 가능하여 박준호 씨의 출장 및 레저 활동에 적합합니다.\n\n2. **삼성카드 S Premier**\n   - **공항 라운지 및 여행자 보험**: 국내외 공항 라운지 이용과 여행자 보험이 포함됩니다.\n   - **자동차 관련 혜택**: 주유 할인 및 자동차 보험료 할인 혜택이 있습니다.\n   - **자녀 교육비 할인**: 교육비 자동이체 시 할인 혜택이 제공됩니다.\n   - **추천 이유**: 가족 중심의 소비 패턴과 자동차 유지비 절감에 도움이 됩니다.\n\n3. **현대카드 M3 Edition2**\n   - **마일리지 적립**: 다양한 항공사 마일리지 적립이 가능하여 해외 출장 시 유리합니다.\n   - **헬스/골프 할인**: 헬스장 및 골프장 이용 시 할인 혜택이 제공됩니다.\n   - **프리미엄 쇼핑 혜택**: 백화점 및 면세점 할인 혜택이 있습니다.\n   - **추천 이유**: 레저 활동과 쇼핑 혜택이 강화되어 있어 박준호 씨의 라이프스타일에 적합합니다.\n\n4. **KB국민카드 The Prestige**\n   - **공항 라운지 및 여행자 보험**: 국내외 공항 라운지 이용과 여행자 보험 혜택이 포함됩니다.\n   - **프리미엄 쇼핑 및 레저 혜택**: 백화점, 면세점, 골프장 등에서 다양한 할인 혜택이 제공됩니다.\n   - **주유소 할인**: 주요 주유소에서 할인 혜택이 있습니다.\n   - **추천 이유**: 전반적인 프리미엄 혜택과 주유 할인으로 박준호 씨의 다양한 소비 패턴을 충족시킵니다.\n\n이 카드들은 박준호 씨의 다양한 필요를 충족시킬 수 있는 혜택을 제공하며, 각 카드의 세부 혜택과 연회비를 비교하여 최적의 선택을 하시길 바랍니다."


#### 일반RetrievalQA

In [279]:
question = f"""당신은 한국 최고의 신용 및 체크 카드 전문가입니다.
{persona1}의 성향과 니즈를 바탕으로 최적의 신용카드와 체크카드를 각각 1개씩 추천해주세요. 
각 카드의 혜택과 추천 이유를 상세하게 설명해주세요. 가능한 한 구체적이고 유용한 정보를 제공해 주시면 좋겠습니다.
아래는 카드 추천 예시입니다. 

{exemple1},
{exemple2},
{exemple3}
위 예시들을 참고하시되 항상 더 좋은 방안은 무엇인지 고려하며 당신의 고객이 누구고 어떠한 사람인지 다시 되세기면서 추천 카드를 제시해주세요.
반드시 한국어로 응답하세요.
"""
print(R_QA1.invoke(question)['result'])

김형석 씨를 위해 추천하는 카드를 다음과 같이 업데이트하겠습니다.

1. **KB국민카드 트래블러스 체크카드**
   - **혜택**: 해외 가맹점 및 ATM 이용 수수료 면제, 국내에서의 혜택으로 KB Pay 200원 추가 할인, 카페 1,000원 할인, 빵집 2,000원 할인, 철도 5,000원 할인, 고속버스 2,000원 할인, 주차장 500원 할인, 전국맛집 5,000원 할인 등 다양한 혜택이 제공됩니다.
   - **추천 이유**: 김형석 씨가 주로 집에서 식사를 하고 클래식 음악을 감상하며 불확실한 미래에 대해 고민하는 취준생으로, KB국민카드 트래블러스 체크카드는 해외 가맹점 및 ATM 이용 시 수수료 면제를 비롯해 국내에서도 다양한 할인 혜택을 제공하여 김형석 씨의 소비 습관과 관심사를 반영한 카드입니다. 또한, 주차장 할인이나 맛집 할인 등 여러 분야에서 혜택을 받을 수 있어 다양한 생활 영역에서 유용하게 사용할 수 있을 것입니다.

KB국민카드 트래블러스 체크카드는 국내외 다양한 혜택을 제공하며, 김형석 씨의 다양한 필요를 고려한 카드로 추천드립니다. 자세한 내용은 [링크](https://www.card-gorilla.com/card/detail/2691)를 참고해 주시기 바랍니다.


In [268]:
question = f"""당신은 한국 최고의 신용 및 체크 카드 전문가입니다.
{persona2}의 성향과 니즈를 바탕으로 최적의 신용카드와 체크카드를 각각 1개씩 추천해주세요. 
각 카드의 혜택과 추천 이유를 상세하게 설명해주세요. 가능한 한 구체적이고 유용한 정보를 제공해 주시면 좋겠습니다.
아래는 카드 추천 예시입니다.

{exemple1},
{exemple2},
{exemple3}

위 예시들을 참고하시되 항상 더 좋은 방안은 무엇인지 고려하며 당신의 고객이 누구고 어떠한 사람인지지 다시 되세기면서 추천 카드를 제시해주세요.
반드시 한국어로 응답하세요.
"""
print(R_QA1.invoke(question)['result'])

신한카드 YOLO Tasty와 함께 KB국민카드의 트래블러스 체크카드를 추천드립니다.

1. **현대카드 MM** (신용카드):
   - **이유**: 국내외 가맹점에서 1.5% M포인트 적립 및 전월 이용 금액에 따른 추가 혜택이 있습니다. 특히, 전월 이용 금액이 50만원 이상이면 온라인 쇼핑몰, 외식, 해외 가맹점에서 10% M포인트 적립이 가능하며, 100만원 이상 이용 시에는 M 긴급적립, 인천국제공항 라운지 무료 이용, 인천공항 발렛파킹 무료 이용 혜택을 누릴 수 있습니다.

2. **KB국민카드 트래블러스 체크카드** (체크카드):
   - **혜택**: 해외 가맹점 및 ATM 이용 수수료 면제를 비롯해 KB Pay 할인, 국내 다양한 할인 혜택을 제공합니다. 특히, 국내 소비에 특화된 할인 혜택과 해외 여행 시에도 편리한 혜택들을 제공하여 다양한 상황에서 혜택을 누릴 수 있습니다. 이러한 혜택은 현대카드 MM과 함께 사용할 때 더욱 효율적으로 소비를 관리할 수 있을 것입니다.

**추가 이유**:
이혜성 씨가 신용카드를 통해 다양한 혜택을 누리고자 한다면, 현대카드 MM을 활용하여 국내외 가맹점에서 M포인트를 적립하는 동시에, KB국민카드 트래블러스 체크카드를 통해 국내 소비 및 해외 여행에 특화된 혜택을 즐기실 수 있을 것입니다. 이를 통해 더욱 다채로운 혜택을 경험하실 수 있습니다.


In [258]:
question = f"""당신은 한국 최고의 신용 및 체크 카드 전문가입니다.
{persona3}의 성향과 니즈를 바탕으로 최적의 신용카드와 체크카드를 각각 1개씩 추천해주세요. 
각 카드의 혜택과 추천 이유를 상세하게 설명해주세요. 가능한 한 구체적이고 유용한 정보를 제공해 주시면 좋겠습니다.
아래는 카드 추천 예시입니다. 

{exemple1},
{exemple3},
{exemple2}

위 예시들을 참고하시되 항상 더 좋은 방안은 무엇인지 고려하며 당신의 고객이 누구고 어떠한 사람인지 다시 되세기면서 추천 카드를 제시해주세요.
반드시 한국어로 응답하세요.
"""
print(R_QA1.invoke(question)['result'])

양호성 씨에게 **IBK기업은행 CEO카드**, **KB국민카드 트래블러스 체크카드**와 **kt SUPER 카드**를 추천드립니다.

**신용카드: IBK기업은행 CEO카드**
- **혜택**: Air Money 포인트 최대 2% 적립, 전세계 공항라운지 무료 이용 서비스, 국내 지정 골프장 무료 커피 제공, 국내 부띠크호텔 브런치세트 무료 제공, 커피점 10% 할인, 온라인 쇼핑 5% 할인, 대형마트 5% 할인, Master Platinum 등급 국제브랜드 서비스 제공, DUAL카드 Service
- **추천 이유**: IBK기업은행 CEO카드는 해외 출장 및 레저 활동에 적합한 혜택을 제공합니다. 공항라운지 무료 이용 서비스와 해외 이용가맹점 추가할인은 해외 여행 시 편리함을 더해줍니다. 또한, 다양한 할인 혜택과 국제브랜드 서비스로 편의성을 높여줄 수 있습니다.

**체크카드: KB국민카드 트래블러스 체크카드**
- **혜택**: 해외 가맹점 및 ATM 이용 수수료 면제, KB Pay 200원 추가 할인, 국내 카페 1,000원 할인, 빵집 2,000원 할인, 철도 5,000원 할인, 고속버스 2,000원 할인, 주차장 500원 할인, 전국맛집 5,000원 할인
- **추천 이유**: KB국민카드 트래블러스 체크카드는 해외 및 국내 혜택을 모두 제공하여 다양한 소비에 유용합니다. 특히, 해외 이용 시에는 수수료 면제와 추가 혜택을 누릴 수 있으며, 국내에서도 다양한 카페, 맛집, 교통수단 등에서 할인 혜택을 받을 수 있습니다.

**신용카드: kt SUPER 카드**
- **혜택**: kt 통신요금 청구할인, 전월실적 별 월 최대 18,000원 할인, SK 주유소 1% 청구할인, 국제 브랜드 서비스, VISA Young Premium 서비스
- **추천 이유**: kt SUPER 카드는 kt 통신요금 청구할인과 주유소 할인 등 다양한 혜택을 제공하여 일상 생활에서의 소비에 유용합니다. 또한, 국제 브랜드 서비스와 VISA Young Premium 서비스

#### ContextualCompressionRetriever 적용

In [ ]:
question = f"""당신은 한국 최고의 신용 및 체크 카드 전문가입니다.
{persona1}의 성향과 니즈를 바탕으로 최적의 신용카드와 체크카드를 각각 1개씩 추천해주세요. 
각 카드의 혜택과 추천 이유를 상세하게 설명해주세요. 가능한 한 구체적이고 유용한 정보를 제공해 주시면 좋겠습니다.
아래는 카드 추천 예시입니다. 

{exemple1},
{exemple2},
{exemple3}
이 예시들을 참고하시되 항상 더 좋은 방안은 무엇인지 고려하며 당신의 고객이 누구고 어떠한 사람인지 다시 되세기면서 추천 카드를 제시해주세요.
반드시 한국어로 응답하세요.
"""
print(R_QA2.invoke(question)['result'])

김형석 씨의 경우를 고려하여 새로운 카드를 추가로 추천드리면:

1. **최적의 신용카드:**
- **신한카드 플리 체크(산리오캐릭터즈)**
  - **혜택**: [Daily 할인] 국내 및 해외 이용금액에 할인 제공, [Weekly 할인 쿠폰] 매월 2회 국내 이용에 할인 쿠폰 제공, [Monthly 단골 적립] My Pick 가맹점 이용 시에 포인트 적립, 카드 플레이트 선택 가능
  - **추가 추천 이유**:
     - 일상적인 소비에 대한 할인 및 적립 혜택이 다양하게 제공됩니다.
     - 카드 플레이트 선택이 가능하여 개성에 맞는 디자인을 선택할 수 있습니다.

2. **최적의 체크카드:**
- **KB국민카드 트래블러스 체크카드**
  - **혜택**: 해외 가맹점, ATM 이용 수수료 면제, KB Pay 추가 할인 및 국내 할인 프로그램 제공
  - **추가 추천 이유**:
     - 해외 이용 및 국내 할인 프로그램을 통해 다양한 혜택을 누릴 수 있습니다.

링크: [KB국민카드 트래블러스 체크카드 상세페이지](https://www.card-gorilla.com/card/detail/2691)

김형석 씨의 다양한 소비 습관과 혜택을 고려해 위의 카드를 추천드립니다.항상 자세한 혜택을 확인하신 후에 최종 선택하시길 바랍니다. 


In [269]:
question = f"""당신은 한국 최고의 신용 및 체크 카드 전문가입니다.
{persona2}의 성향과 니즈를 바탕으로 최적의 신용카드와 체크카드를 각각 1개씩 추천해주세요. 
각 카드의 혜택과 추천 이유를 상세하게 설명해주세요. 가능한 한 구체적이고 유용한 정보를 제공해 주시면 좋겠습니다.
아래는 카드 추천 예시입니다. 

{exemple1},
{exemple2},
{exemple3}

이 예시들을 참고하시되 항상 더 좋은 방안은 무엇인지 고려하며 당신의 고객이 누구고 어떠한 사람인지 다시 되세기면서 추천 카드를 제시해주세요.
반드시 한국어로 응답하세요.
"""
print(R_QA2.invoke(question)['result'])

신협-신한카드 The BEST-F와 KDB산업은행의 KDB Choice 하이브리드 카드를 함께 고려하여 이혜성 님에게 추천드립니다.

**신협-신한카드 The BEST-F (신용카드)**:
   - **이유**: 이 카드는 Gift Option Service, 캐시백, 마일리지 적립, JEJU 할인, 공항 라운지 이용, 커피 할인, 베이커리 할인, 쇼핑 할인, 생활 할인 등의 다양한 혜택을 제공합니다. 또한, 최근 추가된 챔피언 체크카드와 연계된 혜택으로 주요편의점, 커피전문점, 백화점&마트, 온라인 쇼핑몰, 주유소 등에서 10% 캐시백 혜택을 받을 수 있습니다. 이렇게 다양한 혜택을 통해 이혜성 님의 다양한 소비 양식에 맞는 혜택을 제공하며, CMA 서비스를 통해 편리한 은행 거래도 가능합니다.

**KDB Choice 하이브리드 카드 (체크카드)**:
   - **이유**: 이 카드는 10대 주거래 업종 중 고객이 직접 선택한 2개의 혜택 업종에서 전가맹점에서 5% 캐시백 혜택을 제공합니다. 선택 가능한 업종에는 대형마트, 슈퍼, 온라인 쇼핑, 통신, 백화점, 레저, 병의원/약국, 홈쇼핑, 숙박, 커피 등이 포함되어 있습니다. 또한, 후불교통 청구할인 혜택도 제공되어 소비 패턴에 맞게 선택할 수 있습니다.

이 두 가지 카드를 함께 고려할 때, 이혜성 님의 다양한 소비 패턴과 혜택에 맞는 최적의 결합을 찾을 수 있을 것입니다. 두 카드의 혜택과 연동을 통해 효율적인 혜택을 누리며, 소비를 더욱 효율적으로 관리하실 수 있습니다. 자세한 정보는 [여기](https://www.card-gorilla.com/card/detail/1964)에서 확인하실 수 있습니다.


In [276]:
question = f"""당신은 한국 최고의 신용 및 체크 카드 전문가입니다.
{persona3}의 성향과 니즈를 바탕으로 최적의 신용카드와 체크카드를 각각 1개씩 추천해주세요. 
각 카드의 혜택과 추천 이유를 상세하게 설명해주세요. 가능한 한 구체적이고 유용한 정보를 제공해 주시면 좋겠습니다.
아래는 카드 추천 예시입니다. 
{exemple1},
{exemple3},
{exemple2}

이 예시들을 참고하시되 항상 더 좋은 방안은 무엇인지 고려하며 당신의 고객이 누구고 어떠한 사람인지지 다시 되세기면서 추천 카드를 제시해주세요.
반드시 한국어로 응답하세요.
"""
print(R_QA2.invoke(question)['result'])

Based on the additional context provided about the "챔피언 체크카드" from 유진투자증권, we can further refine the recommendation to include this card for consideration due to its attractive benefits such as 10% cashback at major convenience stores, coffee shops, department stores, online shopping malls, and gas stations. Here is the updated recommendation:

1. **신한카드 플리 체크(산리오캐릭터즈)**
   - **혜택**:
     - [Daily 할인] 국내 이용금액 최대 0.3% 할인, 해외 이용금액 0.2% 할인
     - [Weekly 할인 쿠폰] 매월 2회 국내 이용 할인 쿠폰 제공
     - [Monthly 단골 적립] My Pick 가맹점 월 3, 6, 9번째 이용 건에 대해 마이신한포인트 1,000포인트 적립
     - [선택] 카드 플레이트 설정 가능
   - **추가 이유**:
     - 다양한 할인 혜택과 쿠폰 제공으로 일상적인 소비에 유용함
     - 월간 적립 혜택을 통해 소비 패턴에 맞춰 포인트를 모을 수 있음
     - 캐릭터 디자인의 카드 플레이트 선택 가능으로 개성을 표현할 수 있음

2. **챔피언 체크카드 from 유진투자증권**
   - **혜택**:
     - [기본혜택] 주요편의점 10%캐시백, 주요커피전문점 10%캐시백
     - [선택형 캐시백 혜택] 백화점&마트, 온라인쇼핑, 주유 캐시백 형태로 다양한 선택 가능
     - [부가혜택] CMA 서비스 제공으로 은행 CD/ATM 출금거래, 체크카드 거래 혜택 제공
   - **추가 이유**:
     - 다양한 업종에서 높은 캐시백 혜택을 제공하여 소비 다양화 가능
     - 부가혜택으로 C

In [ ]:
# personas = [persona1, persona2, persona3]

# for persona in personas:
#     question = f"""당신은 한국 최고의 신용 및 체크 카드 전문가입니다.
# {personas}의 성향과 니즈를 바탕으로 최적의 신용카드와 체크카드를 각각 1개씩 추천해주세요. 
# 각 카드의 혜택과 추천 이유를 상세하게 설명해주세요. 가능한 한 구체적이고 유용한 정보를 제공해 주시면 좋겠습니다.
# 아래는 카드 추천 예시입니다. 

# {exemple1},
# {exemple2},
# {exemple3}

# 이 예시들을 참고하시되 항상 더 좋은 방안은 무엇인지 고려하며 당신의 고객이 누구인지 어떠한 사람인지지 다시 되세기면서 추천 카드를 제시해주세요.
# 반드시 한국어로 응답하세요.
# """
#     print(R_QA2.invoke(question)['result'])
#     print("*"*100)

**신한카드 플리 체크(산리오 캐릭터즈)** 대신, 박경화 씨에게는 **S1카드**를 추천드립니다.

1. **S1카드** (신용카드):
   - **이유**: S1카드는 Sh수협은행에서 제공하는 카드로, 다양한 업종에서 7% 청구 할인 혜택을 제공합니다. 특히, 선택한 업종에서 7% 청구 할인과 리터당 60원의 주유소 할인 혜택이 있어 다양한 소비에 부합할 것입니다. 또한, 전국의 주요 가맹점에서 다양한 할인 혜택을 받을 수 있어 신용 및 체크카드 전문가로서 다양한 소비에 활용할 수 있을 것입니다.
   - **혜택**: 7% 청구 할인(다양한 업종 선택 가능), 리터당 60원 주유소 할인
   - **관련 링크**: [S1카드 바로가기](https://www.card-gorilla.com/card/detail/1961)

S1카드는 다양한 소비 분야에서 혜택을 누릴 수 있는 카드이며, 특히 주요 가맹점에서의 할인 혜택이 다양하게 제공되어 박경화 씨의 다양한 소비 습관에 부합할 것으로 판단됩니다.문의사항이 있으시면 언제든지 물어보세요.
****************************************************************************************************
신한카드 The BEST-F를 추천해 드립니다.

1. **신한카드 The BEST-F** (신용카드)
   - **혜택 및 이유**:
     - **Gift Option Service**: 다양한 상품 및 서비스로 교환 가능한 기프트 옵션 서비스를 제공하여 좋은 선물을 선택할 수 있습니다.
     - **캐시백 또는 마일리지 적립**: 캐시백 형태 또는 마일리지 적립 형태 중 선택 가능하여, 원하는 혜택을 선택할 수 있습니다.
     - **JEJU 할인**: JEJU항공 티켓 예약 시 할인 혜택을 받을 수 있습니다.
     - **공항라운지**: 국내외 공항 라운지 무료 이용 혜택이 제공됩니다.
     - **할인 혜택**: 커피, 베이